#### Importing libraries


import pandas as pd
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt
import nltk


# importing dataset as fake and real

In [4]:
data_fake = pd.read_csv('fake.csv')
data_real = pd.read_csv('real.csv')

NameError: name 'pd' is not defined

In [ ]:
data_real.head()

In [ ]:
data_fake.head()

# clasify gareu data fake lai 0 le real lai 1 lw

In [ ]:
data_fake["class"]= 0
data_real["class"]= 1

# 2 tai data lai concatinate gareu

In [ ]:
data = pd.concat([data_real,data_fake],axis=0)
data.head()

# droping the column named description

In [ ]:
cleaned_data= data.drop(['Description'],axis=1, inplace=False)
cleaned_data.head(10)

# suffle gareu data lai

In [ ]:
suffle_data = cleaned_data.sample(frac=1)
suffle_data.head(10)

# Adding new index

In [ ]:
# due to suffleing the index also gets suffeled.
# So reseting new index of the dataframe to its default integer_based indexing.
suffle_data.reset_index(inplace=True)
suffle_data.head()

In [ ]:
# deleting the old index column
suffle_data.drop(['index'],axis=1,inplace = True)
suffle_data.head(5)

# checking null value

In [ ]:
suffle_data.isnull().sum()

# mixing both fake and real into 1

In [ ]:
suffle_data.to_csv('mixed.csv', index=False)

In [ ]:
data = pd.read_csv('mixed.csv')
data.head()

# Preprocessing

In [ ]:
def wordopt(Title):
    
    
    # Title = Title.lower()
    Title = re.sub(r'[\[\]]', '', Title)  # Remove square brackets
    Title = re.sub(r'https?://\S+|www\.\S+', '', Title)  # Remove URLs
    Titel = re.sub(r'<.*?>', '', Title)  # Remove HTML tags
    Title = re.sub(f'[{re.escape(string.punctuation)}]', '', Title)  # Remove punctuation
    Title = re.sub(r'\n', '', Title)  # Remove newlines
    Title = re.sub(r'\w*\d\w*', '', Title)  # Remove words with digits
    Title = re.sub(r'\b[a-zA-Z]+\b', '', Title)  # This removes English words    
    return Title


In [ ]:
data['Title2']= data['Title'].apply (wordopt)
data.head(50)

In [ ]:
data.columns

# Tokenization

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
data['tokens1'] = data['Title2'].apply(word_tokenize)
data.head()

# acessing the list of stopwords

In [ ]:
stop = open('stopwords.txt', encoding='utf-8')  # Specify encoding
stopwords = []
for x in stop: # perform through each line of the txt file ma.
    x = x.replace('\n', '')  # Remove newline characters
    stopwords.append(x)      # Add to the stopwords list
stop.close()  # Close the file after reading
data.head(1)

In [ ]:
stopwords

# Removing stopwords

In [ ]:
def remove_stopwords(x):
    return ''.join([word for word in str(x).split() if word not in stopwords])
data['title3']= data['tokens1']. apply(lambda x :remove_stopwords(x))# for each row in the tokens column 
#lambda function is called with x being the value of that row.
#The result of the lambda function (which is the value returned by remove_stopwords(x)) is assigned to the new column title3.
data.head(5)

# Stemming the language

In [ ]:
# import snowballstemmer
# test_words = ["नेपालीको", "आफ्नै", "स्तेम्मेर", "यसलाई", "अनुमोदनको"]
#  stemmed_words = nepali_stemmer.stemWords(test_words)
#  print("Original Words:", test_words)
#  print("Stemmed Words:", stemmed_words)


In [ ]:
# import re
# import snowballstemmer

# # Initialize the NepaliStemmer instance
# nepali_stemmer = snowballstemmer.NepaliStemmer()

# def preprocess_text(text):
#     if isinstance(text, str):
#         # Remove punctuation and extra spaces
#         text = re.sub(r'[^\w\s]', '', text)
#         return text.strip()
#     return text

# def stem_text(text):
#     text = preprocess_text(text)
#     if isinstance(text, str):
#         return " ".join(nepali_stemmer.stemWords(text.split()))
#     return text

# # Apply the preprocessing and stemming to the 'title3' column
# data['data4'] = data['title3'].apply(stem_text)

# # Display results
# print(data[['title3', 'data4']].head(20))
# data.head(20)

In [ ]:
data.columns

In [ ]:
x = data['title3']
y = data['class']

In [ ]:
x

In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.10)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_train.head(50)

In [ ]:
y_test.head(50)

In [ ]:
# here i built different file which contain only testing data
x_test.to_csv('testing_data.csv', index=False)

In [ ]:
# similarly building file for training data
x_train.to_csv('training_data.csv', index=False)

# Vectorizing the text data into numerical


In [ ]:
# #  here due to the nepali datasets the system doest tfidf the data.
# # so for that i have to convert the data into numerical featues using countvectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(ngram_range=(1,2)).fit(x_train)
# x_train_vectorized = vectorizer.transform(x_train)
# print(x_train_vectorized)
# x_train_vectorized.shape

In [ ]:
# vectorizer = CountVectorizer(ngram_range=(1,2))
# x_train_vectorized = vectorizer.fit_transform(x_train)

# Transforming the text data

In [ ]:
#converting the text into vector The TfidfVectorizer is used to convert a collection of text documents into numerical vectors,
#that can be fed into machine learning models. It does this by calculating the TF-IDF for each word in the documents.
vectorization = TfidfVectorizer() 
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
xv_train

# Model Training

In [ ]:
model = LogisticRegression()
model.fit(xv_train,y_train)

In [ ]:
pred_lr=model.predict(xv_test)

In [ ]:
model.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_lr))

In [ ]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"

def manual_testing(news):
    # Create a DataFrame from the input news
    testing_news = {"title3": [news]}
    new_def_test = pd.DataFrame(testing_news)

    # Apply preprocessing using wordopt (assuming it's a defined function)
    new_def_test["title3"] = new_def_test["title3"].apply(wordopt) 

    # Transform the text using vectorization
    new_x_test = new_def_test["title3"]
    new_xv_test = vectorization.transform(new_x_test)

    # Perform prediction using Logistic Regression
    pred_model = model.predict(new_xv_test)

    # Output the prediction result
    return print("\n\nLogistic Regression Prediction: {}".format(output_label(pred_model[0])))


In [ ]:
news_input = "['उफ्','मिडियाले','झूट','बोल्योसैन्य','सदस्यहर..."
manual_testing(news_input)

# Confusion metrics

In [ ]:
from sklearn.metrics import confusion_matrix

# confusion metrics
confusion_matrix(y_test, pred_lr,labels=[0, 1])

# saving the model using pickle

In [ ]:
import pickle


In [ ]:
# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

In [ ]:
# load saved model
with open('model_pkl' , 'rb') as file:
    model= pickle.load(file)

# vectorized form ma lageko 

In [ ]:
input_text = ["उफ्','मिडियाले','झूट','बोल्योसैन्य','सदस्यहर.."]
input_vectorized = vectorization.transform(input_text)

In [ ]:
# model.predict([["'राष्ट्रपति','पदका','उम्मेद्वार','मार्को','रु.."]])
prediction = model.predict(input_vectorized)
print(prediction)